In [2]:
import numpy as np

from sklearn.linear_model import MultiTaskLasso, Lasso, ElasticNet
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
from tqdm import tqdm

In [2]:
in_seq_len = 7
out_seq_len = 6
out_freq = 5
days = 164
daily_slots_5_min = 180
daily_slots_1_min = 900
seed = 42

input = np.load("../data/new_X.npy")  # (29520, 1796)
output_tmc = np.load("../data/new_Y_tmc.npy")[..., :3]  # (29520, 70, 3)
output_xd = np.load("../data/new_Y_xd.npy")[..., 0]  # (147600, 70)

In [ ]:
# prepare X
X = []
for d in range(days):
    for i in range(daily_slots_5_min-in_seq_len+1):
        X.append(input[d*days+i : d*days+i+in_seq_len])
X = np.stack(X, axis=0) # (28536, 7, 1796)

In [ ]:
# prepare Y_xd
Y_xd = []
for d in range(days):
    for i in range(1, daily_slots_5_min-out_seq_len+1):
        Y_xd.append(output_xd[[d*days+j*out_freq for j in range(i, i+out_seq_len)],...])
Y_xd = np.stack(Y_xd, axis=0) # (28536, 6, 70)